
#AMA Kewyword Cleaner & Keyword Clustering Tool V3

- tag by intent, location
- simple spell check and flag for review
- simple language check to flag non-English rows
- OPT: removes duplicate keywords (URL insensitive)
- removes negative KWs (based on chosen dictionary/relevant BU)


---



***NEXT: choose custom intent classifiers & clustering by dictionary/BU***

***NEXT: support all upload types from Conductor***



---


## Quick Start Instructions
Run all the cells and upload a CSV export.
Runtime > Run All (Control + F9)

### Works with the Following Exports out the Box

*   Ahrefs.com (Keyword Export / Site Explorer Export)
*   SEMRush.com
*   Search Console (Coverage Report CSV Export (Queries.csv))
*   AdWords Search Terms Report .csv or Excel format (Beta)
*   A simple single column .txt / csv file with keywords (Header or Headerless)

### File Formats
*   utf-8/utf-16/csv/xls/xlsx/xlsm/xlsb/odf/ods/odt

In [2]:
!pip install pandas
!pip install polyfuzz[fast]
!pip install chardet
!pip install tqdm
!pip install langdetect
!pip install pyspellchecker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=05560344a992c0d39235b233e43a7f3293cd7d8c86e3bc5759bdd88435d48748
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 31.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import sys
from google.colab import files
from polyfuzz import PolyFuzz
import chardet
from tqdm import tqdm
import os
from langdetect import detect
from langdetect import DetectorFactory
from spellchecker import SpellChecker
# Set the langdetect factory seed for reproducibility
DetectorFactory.seed = 0


In [462]:
# rename the parent cluster name using the keyword with the highest search volume (recommended)
parent_by_vol = True
drop_site_links = False
drop_image_links = False
sim_match_percent = 1
url_filter = ""
min_volume = 0  # set the minimum search volume / impressions to filter on

## **Define Negative Keywords to Clean**

In [463]:
negkw_sets = {
    "insurance": "lawyer|sue|lawsuit",
    "registries": "girl|signin|login|sign in|log in|portal|www|http|link|adopt|dog|cow|rodeo|vision|cerb|poppy|shoe|barber|chef|gold|sin ",
    "DE": "claim|lawyer",
    # Add more sets as needed
}

In [464]:
nonabkw_filter = "^bc | bc | bc$|^sk | sk | sk$| ns$|bathurst|afric|manitoba|quebec|^us | us$| us |halifax|markham| mb$|^mb |mexico|mexican|^ns |manitoba|quebec|nova scotia|^ns |markham|hst|pst|qst|^sk |^nl |^nb | ou | is$| in$| a$|^us | us$|california|2015|2016|2017|2018|2019|2020|2021|2022|nwt|^a |america|barrie|brampton|british|brunswick|fredericton|halifax|hamilton|hartford|kitchener| la$|manitoba|michigan|mile|missisauga|mkmk|moncton|montreal|newfoundland| nl|ontario|ottawa|pei|plano|prince|quebec|regina|sask|nova|spokane|state|sudbury|toronto|troy|tx|vancouver|waterloo|windsor|winnipeg| us$|^us |sturgeon|qc|pq"

In [465]:
commonlymissedfr_filter = "incorporer|livre|marque|marques|domaine|registraire|capitale|canadienne|societe|sans|registre|é|à"

## **Define Intent Classifiers**

In [466]:
info_filter = "what|where is|where does|why|when|who|how does|how to|which|tip|reddit|guide|tutorial|ideas|example|learn|wiki|in mm|in cm|in ft|in feet|question"
comm_invest_filter = "best| vs|list|compare|review|list|^top|difference between|alternative|competitor|case study|rating|^rank|plan|company|companies|advis|agen"
trans_filter = "affordable|purchase|bargain|cheap|deal|value|buy|shop|coupon|discount|price|pricing|order|sale|cost|how much|estimat|quote|rate|calculat|get|add "

## **Define Custom Cluster Roots**

In [491]:
alberta_filter = "^ab |alberta|near|local|edmonton|calgary|lethbridge|camrose|mcmurray|grande|prairie|medicine|deer|sherwood|albert$|willow|kingsway|sunridge|manning|crowfoot|shawnessy|airdrie|leduc|ponoka|wetask|hythe|spruce|okotok|high river"
nearme_filter = "near me|local"
cities_filter = "edmonton|calgary|lethbridge|camrose|mcmurray|grande|prairie|medicine|deer|sherwood|albert$|willow|kingsway|sunridge|manning|crowfoot|shawnessy"
neighbor_filter = "airdrie|leduc|ponoka|wetask|hythe|spruce|okotok|high river"
branded_filter = "ama|caa|aaa|motor association"
insurance_filter = "accept|covered|claim| no |without"
registry_filter = "cvip|marriage|abstract|handicap| ahs|^ahs |carfax|notar|lein|lien|^lean|vin|^id | id |birth|certificate|passport|identification|incorpor|learner|test|plate|inspect||class |gdl|corpor|dissolu|bill |black|shred|health care card|health card|registr|license|fine|"
de_filter = "driv|road test|train|practice|quiz|instruct|lesson|school|fail|parallel|"
vehicle_filter = "auto|car |truck|vehicle|motorcycle|bike|atv|boat|moped|snowmobile|motor insurance|motor car|car motor| car$| cars$|scooter|motors|plpd|fault"
property_filter = "home|house|condo|modular|rv|recreational|motorhome|trailer|vacation|tennant|tenant|rent|apartment|property|content|antique"
pet_filter = "pet|cat|dog|puppy|animal|feline"
health_filter = "life|health|dental|accident|disability|illness|even|medical|wedding|extended|injur|benefit"
commercial_filter = "business|commercial|work|liab|agri|farm|corporat"
partner_filter = "depot|fountain|freshco|husky|^bills|grocery|gift card|keg|kernel|chatters|master|reload| inn|creative door| 60|marmot|pita|f2|fionn|enterprise|edible|mario|dulux|^don|derrick|derk|73|pizza|clover|eurocraft|staples|zoo|tutti|mobil|promo|lenscraft|pennington|101|mobility|a 1|academy hear|acadia|auto service|access automotive|accurate transmission|ac hotels|action muffler|best western|alamo|alberta driveline|marine|massage|aldo|alexanian|carpet|flooring|all lock rescue ltd|aloft|altitude sports|amj campbell|amsteam carpet cleaning inc|andaz|anytime fitness|a r automotive|ardene|ashley homestore|autograph collection|auto imports|auto trac alignment|babycakes cupcakery|bento sushi|best choice automotive|best western|best western hotels resorts gift card|best western plus|best western premier|bills garage|bliss yoga spa ltd|blondies gift garden ltd|^bou|bowl alberta|boyd royal vista|bravado designs|browns chrysler|bruce stewarts auto repair centre|bumble and bumble|bw premier collection|caddy auto service|calgary lock safe|calgary renovation show|calgary zoo|canada golf card|canadian rockies hot springs|canadream|canmore river adventures ltd|canon estore canada|canopy|cantrust auto repair|can west transmissions parts|resort|care cleaners|carstar|carters|oshkosh|castle toys|castrol|centre street|century motors|certapro|certified radio|cetus automotive|chatters|chopped leaf|cineplex|city collision|city wide|radiator|clarins|cleaningpros|clean x carpet upholstery cleaning ltd|clinique|cloverdale paint|coast appliances|columbia sportswear|computerized autopro|connect hearing|conrad hotels resorts|contemporary coachworks|cookies by george|cookies by george gift card|country automotive specialists|courtyard by marriott|cozey|creative door services ltd|crowfoot minit tune brake|csn collision centres|cub cadet|curio|dale adams automotive|dalhousie auto service|dandy auto marine rv ltd|davidstea|deco windshield repair|dell|delta|deluxe rv service sales|denham ford sales|derks formals|derrick dodge|dignity memorial|dons tire automotive repair|dot transmissions|doubletree hotels|dougs|dulux|earls|eastern mechanics|eastside dodge chrysler jeep ram fiat|east side marios|edible arrangements|edition|edmonton elks|edmonton renovation show|edmonton wedding party centre|element|elite|marketing|embassy suites|enterprise|enterprise truck rental|escape 60|ethos bridal group|eurocraft collision center calgary ltd|evergreen lawn services|executive residency by best western|expressions at home|extreme pita|f2 furnishings|fabricville|fairfield inns|fairfield inn suites|fairmont hotels resorts|fionn maccools|first choice collision|fishmans care cleaners|fix auto|flirt cupcakes|foothills mechanical services|forever 21|fort calgary|fort george and buckingham house provincial historic site|fountain tire calgary bowness|four points by sheraton|frank slide interpretive centre|freedom ford|freshii|fusion collision|garage 104|gate ave service|gaylord hotels|gershaw auto|glassmasters autoglass ltd|glo|globo shoes|godiva|go rv|grande prairie auto repair|grand hyatt|great canadian oil change|grower direct alberta|gm|hampton|hart|harveys|buffalo|herbers|hilton|dunvegan|hi tech|home2|home hardware|homewood|horton|h r|hudsons|hyatt|inabuggy|indigo|infinity|innisfail|integra|international|irobot|it cosmetics|jacks|j adams autobody|jamieson|vitamins|jasper raft|skytram|jds|jm|joe fresh|jugo juice|just junk|jw marriott|kal tire|kernels gift card|keurig|kingsgate automotive|kingsway toyota|k js custom granite inc|kleiber automotive|knibbe automotive repair|koch ford lincoln|kudos for wood furniture|lacombe auto service|lake louise ski resort and summer gondola|lakeview husky|lamb ford sales|lanco me|landmark cinemas|lego|le meridien|lenscrafters|linen chest|loblaw optical|lole|londonderry auto service|londonderry collision|lone wolf mechanical|lube city|lube city gift card|luxury collection|macleod auto truck repair|malik auto centre|mancuso carpet and upholstery cleaning|maranello auto refinishing ltd|marble slab creamery|marmot basin|marriott hotels and resorts|master cleaner|matt nat|mcmaster|mgm|lube|millwoods auto inspection repairs|ming shine|mint auto service|minute muffler|mi vida massage and wellness|mobil 1 lube express|monster mini golf|montanas|monza auto|morinville tirecraft|mount pleasant auto service|moxy hotels|mri autocare|mr lube|mudders wash|napa|national|national transmission|natura market|new asian village|new york fries|nitehawk year round adventure park|northgate chevrolet buick gmc ltd|northgate transmission centre|north hill auto service|nutri lawn|nyx professional makeup|oasis tirecraft|oil sands discovery centre|ok tire|optimum auto service|original joes restaurant bar|origins|paris jewellers|paris jewellers gift card|park2go|park2go airport rv parking self storage|park hyatt|parkland nurseries and garden centre|park n fly|parks canada|pedego electric bikes|penningtons|penske truck rental|pets plus us pet insurance|pioneer acres museum|pirelli tires inc|pizza 73|pods|poko popcorn|presidents choice|prfo|puzzle|rally|rapid brake|superstore|rec room|red arrow|reddy|reitmans|reliance|remedysrx|remington|carriage|renaissance|residence inn|reynolds alberta museum|richs garage|r j services ltd|roadrunners equipment|rocky mountain chocolate|rockys battery|rooster cafe kitchen|roots|royal canadian mint|running room gift card|rutherford|rw co|rwco|ryanco automotive|sacred arts wellness|sadie best western|samsung|sephora|sheraton hotels resorts|sherlocks automotive repair|sherwood flooring|shoppers drug mart beauty|shoppers drug mart gift card|shopping ca gift card|simons|simplicity car care calgary south|snow auto|soak luxury bath|sobeys safeway freshco iga gift card|southern autobody inc|southgate volkswagen|southwest auto service|sparkling hill resort|special event rentals|speedy apollo|speedy auto service|springhill suites|spruceland truck trailer|stallion van lines|staples|staples in store|state main kitchen bar|stephansson house provincial historic site|st john ambulance|stokes|st regis hotels resorts|sturgeon auto body shop|subaru|superior paint|suzy shier|swiss chalet|tapestry|teleflora|telus|science centre|telus world of science edmonton|the costume shoppe|the fairmont hotel macdonald|the globe and mail|the hangar flight museum|the hudsons bay company|the keg gift card|the last hunt|the right price auto|the ritz carlton|sherlock|the source|ultimate dining|think kitchen|thorncliffe|tiffanys|steak|tile town|tire kings|tolley tire|tom gerrys|total auto care|towneplace suites|trades automotive|trail tire auto centers|tribute|troubled monk|troy bilt|tutti frutti breakfast lunch|twilite music services|uber eats gift card|uber gift card|ukrainian cultural heritage village|vegreville mechanical|via rail|vib|vichy|vics service|victoria settlement provincial historic site|village auto tire services|vinces auto care|waldorf|walmart|tirecraft|warrens automotive|weddingstar|we kare|well ca|west end tire|westin|westside automotive|white s tire |w hotel|whyte|museum|wok|chevrolet|wolfe|hotel|collection|yardly|yoga|you4ia|^oj|combo|lasthint|via|horton|sephora|delta|^kal|nutri |gershaw|shell"

## **Upload File, Toggle De-Duping & Select Neg KW Set**

In [492]:
# upload the keyword export
upload = files.upload()
input_file = list(upload.keys())[0]  # get the name of the uploaded file
# test the file extension
file_extension = os.path.splitext(input_file)

Saving appended-competitor-data.xlsx to appended-competitor-data.xlsx


In [493]:
# ---------------------------------- auto detect character encoding ----------------------------------------------------

with open(input_file, 'rb') as rawdata:
    result = chardet.detect(rawdata.read(10000))

# if the encoding is utf-16 use a space separator, else ','
if result['encoding'] == "UTF-16":
    white_space = True
else:
    white_space = False

if (
    file_extension[1] == ".xlsx"
    or file_extension[1] == ".xls"
    or file_extension[1] == ".xlsm"
    or file_extension[1] == ".xlsb"
    or file_extension[1] == ".odf"
    or file_extension[1] == ".ods"
    or file_extension[1] == ".odt"
):
    df_1 = pd.read_excel(input_file, engine="openpyxl")
else:
    try:
        df_1 = pd.read_csv(
            input_file,
            encoding=result["encoding"],
            delim_whitespace=white_space,
            error_bad_lines=False,
        )
    # fall back to utf-8
    except UnicodeDecodeError:
        df_1 = pd.read_csv(
            input_file,
            encoding="utf-8",
            delim_whitespace=white_space,
            error_bad_lines=False,
        )


In [494]:
# -------------------------- check if single column import / and write header if missing -------------------------------

# check the number of columns
col_len = len(df_1.columns)
col_name = df_1.columns[0]

if col_len == 1 and df_1.columns[0] != "Keyword":
    df_1.columns = ["Keyword"]

if col_len == 1 and df_1.columns[0] != "keyword":
    df_1.columns = ["Keyword"]


In [495]:
response = input("Remove duplicate keywords? (y/n): ")
if response.lower() == 'y':
    df_1 = df_1.drop_duplicates(subset=['Keyword'], keep='first')

Remove duplicate keywords? (y/n): y


In [496]:
def select_negkw_set():
    print("Select a set of negative keywords:")
    for i, key in enumerate(negkw_sets.keys()):
        print(f"{i + 1}. {key}")

    choice = input("Enter the number of your choice: ")
    try:
        selected_key = list(negkw_sets.keys())[int(choice) - 1]
        return negkw_sets[selected_key]
    except (IndexError, ValueError):
        print("Invalid selection. Please enter a valid number.")
        return select_negkw_set()


In [497]:
# Get the selected set of negative keywords
selected_negkw_filter = select_negkw_set()

Select a set of negative keywords:
1. insurance
2. registries
3. DE
Enter the number of your choice: 2


In [498]:
# -------------------------- detect if import file is adwords and remove the first two rows ----------------------------
adwords_check = False
if col_name == "Search terms report":
    df_1.columns = df_1.iloc[1]
    df_1 = df_1[1:]
    df_1 = df_1.reset_index(drop=True)

    new_header = df_1.iloc[0]  # grab the first row for the header
    df_1 = df_1[1:]  # take the data less the header row
    df_1.columns = new_header  # set the header row as the df header
    adwords_check = True

In [499]:
# --------------------------------- Check if csv data is gsc and set bool ----------------------------------------------

if 'Impressions' in df_1.columns:
    gsc_data = True


In [500]:
# ----------------- standardise the column names between ahrefs v1/v2/semrush/gsc keyword exports ----------------------

df_1.rename(
    columns={
        "Current position": "Position",
        "Current URL": "URL",
        "Current URL inside": "Page URL inside",
        "Current traffic": "Traffic",
        "KD": "Difficulty",
        "Keyword Difficulty": "Difficulty",
        "Search Volume": "Volume",
        "page": "URL",
        "query": "Keyword",
        "Top queries": "Keyword",
        "Impressions": "Volume",
        "Clicks": "Traffic",
        "Search term": "Keyword",
        "Impr.": "Volume",
        "Search vol.": "Volume",
    },
    inplace=True,
)

In [501]:
# Define a function to detect language
def detect_language(text):
    try:
        return detect(text)
    except:
        return "error"  # Returns 'error' if language detection fails

# Apply the language detection function to the 'Keyword' column
df_1['lang'] = df_1['Keyword'].apply(detect_language)

# Filter out rows that are not detected as English
#df_1 = df_1[df_1['lang'] == 'en']

# Optionally, you can drop the 'lang' column if you no longer need it
# df_1 = df_1.drop(columns=['lang'])




In [502]:
# ------------------------------ check number of imported rows and warn if excessive -----------------------------------

row_len = len(df_1)
if col_len > 1:
    # --------------------------------- clean the data pre-grouping ----------------------------------------------------

    if url_filter:
        print("Processing only URLs containing:", url_filter)

    try:
        df_1 = df_1[df_1["URL"].str.contains(url_filter, na=False)]
    except KeyError:
        pass

    # ========================= clean strings out of numerical columns (adwords) ========================================

    try:
        df_1["Volume"] = df_1["Volume"].str.replace(",", "").astype(int)
        df_1["Traffic"] = df_1["Traffic"].str.replace(",", "").astype(int)
        df_1["Conv. value / click"] = df_1["Conv. value / click"].str.replace(",", "").astype(float)
        df_1["All conv. value"] = df_1["All conv. value"].str.replace(",", "").astype(float)
        df_1["CTR"] = df_1["CTR"].replace(" --", "0", regex=True)
        df_1["CTR"] = df_1["CTR"].str.replace("\%", "").astype(float)
        df_1["Cost"] = df_1["Cost"].astype(float)
        df_1["Conversions"] = df_1["Conversions"].astype(int)
        df_1["Cost"] = df_1["Cost"].round(2)
        df_1["All conv. value"] = df_1["All conv. value"].astype(float)
        df_1["All conv. value"] = df_1["All conv. value"].round(2)

    except Exception:
        pass

    df_1 = df_1[~df_1["Keyword"].str.contains("Total: ", na=False)]  # remove totals rows
    df_1 = df_1[~df_1["Keyword"].str.contains("td", na=False)]  # remove TD rows
    df_1 = df_1[~df_1["Keyword"].str.contains(selected_negkw_filter, na=False)]  # remove neg kw rows
    df_1 = df_1[~df_1["Keyword"].str.contains(nonabkw_filter, na=False)]  # remove non-AB kw rows
    df_1 = df_1[~df_1["Keyword"].str.contains(commonlymissedfr_filter, na=False)]  # remove French kw rows the lang check seems to miss
    df_1 = df_1[df_1["Keyword"].notna()]  # keep only rows which are NaN
    df_1 = df_1[df_1["Volume"].notna()]  # keep only rows which are NaN
    df_1["Volume"] = df_1["Volume"].astype(str)
    df_1["Volume"] = df_1["Volume"].apply(lambda x: x.replace("0-10", "0"))
    df_1["Volume"] = df_1["Volume"].astype(float).astype(int)

    # drop sitelinks

    if drop_site_links:
        try:
            df_1 = df_1[~df_1["Page URL inside"].str.contains("Sitelinks", na=False)]  # drop sitelinks
        except KeyError:
            pass
        try:
            if gsc_data:
                df_1 = df_1.sort_values(by="Traffic", ascending=False)
                df_1.drop_duplicates(subset="Keyword", keep="first", inplace=True)
        except NameError:
            pass

    if drop_image_links:
        try:
            df_1 = df_1[~df_1["Page URL inside"].str.contains("Image pack", na=False)]  # drop image pack
        except KeyError:
            pass

    df_1 = df_1[df_1["Volume"] > min_volume]

# start strip out all special characters from a column
spec_chars = ["!",'"',"#","%","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–"]
for char in spec_chars:
    df_1['Keyword'] = df_1['Keyword'].str.replace(char, ' ')

In [503]:
# ------------------------------------- do the grouping ----------------------------------------------------------------

df_1_list = df_1.Keyword.tolist()  # create list from df
model = PolyFuzz("TF-IDF")

cluster_tags = df_1_list[::]
cluster_tags = set(cluster_tags)
cluster_tags = list(cluster_tags)

print("Cleaning up the cluster tags.. Please be patient!")
substrings = {w1 for w1 in tqdm(cluster_tags) for w2 in cluster_tags if w1 in w2 and w1 != w2}
longest_word = set(cluster_tags) - substrings
longest_word = list(longest_word)
shortest_word_list = list(set(cluster_tags) - set(longest_word))

try:
    model.match(df_1_list, shortest_word_list)
except ValueError:
    print("Empty Dataframe, Can't Match - Check the URL Filter!")
    sys.exit()

model.group(link_min_similarity=sim_match_percent)
df_matched = model.get_matches()

Cleaning up the cluster tags.. Please be patient!


100%|██████████| 8910/8910 [00:08<00:00, 1046.35it/s]


In [504]:
# ------------------------------- clean the data post-grouping ---------------------------------------------------------

df_matched.rename(columns={"From": "Keyword", "Group": "Cluster Name"}, inplace=True)  # renaming multiple columns

# merge keyword volume / CPC / Pos / URL etc data from original dataframe back in
df_matched = pd.merge(df_matched, df_1, on="Keyword", how="left")

# rename traffic (acs) / (desc) to 'Traffic for standardisation
df_matched.rename(columns={"Traffic (desc)": "Traffic", "Traffic (asc)": "Traffic", "Traffic potential": "Traffic"}, inplace=True)

if col_len > 1:

    # fill in missing values
    df_matched.fillna({"Traffic": 0, "CPC": 0}, inplace=True)
    df_matched['Traffic'] = df_matched['Traffic'].round(0)
    # ------------------------- group the data and merge in original stats -------------------------------------------------
    if not adwords_check:
        try:
            # make dedicated grouped dataframe
            df_grouped = (df_matched.groupby("Cluster Name").agg(
                {"Volume": sum, "Difficulty": "median", "CPC": "median", "Traffic": sum}).reset_index())
        except Exception:
            df_grouped = (df_matched.groupby("Cluster Name").agg(
                {"Volume": sum, "Traffic": sum}).reset_index())

        df_grouped = df_grouped.rename(
            columns={"Volume": "Cluster Volume", "Difficulty": "Cluster KD (Median)", "CPC": "Cluster CPC (Median)",
                     "Traffic": "Cluster Traffic"})

        df_matched = pd.merge(df_matched, df_grouped, on="Cluster Name", how="left")  # merge in the group stats

    if adwords_check:

        df_grouped = (df_matched.groupby("Cluster Name").agg(
            {"Volume": sum, "CTR": "median", "Cost": sum, "Traffic": sum, "All conv. value": sum, "Conversions": sum}).reset_index())

        df_grouped = df_grouped.rename(
            columns={"Volume": "Cluster Volume", "CTR": "Cluster CTR (Median)", "Cost": "Cluster Cost (Sum)",
                     "Traffic": "Cluster Traffic", "All conv. value": "All conv. value (Sum)", "Conversions": "Cluster Conversions (Sum)"})

        df_matched = pd.merge(df_matched, df_grouped, on="Cluster Name", how="left")  # merge in the group stats

        del df_matched['To']
        del df_matched['Similarity']

    # ---------------------------- clean and sort the final output -----------------------------------------------------

    try:
        df_matched.drop_duplicates(subset=["URL", "Keyword"], keep="first", inplace=True)  # drop if both kw & url are duped

    except KeyError:
        pass

In [505]:
if not adwords_check:
    cols = (
        "Cluster Name",
        "Keyword",
        "Cluster Size",
        "Cluster Volume",
        "Cluster KD (Median)",
        "Cluster CPC (Median)",
        "Cluster Traffic",
        "Volume",
        "Difficulty",
        "CPC",
        "Traffic",
        "URL",
    )

    df_matched = df_matched.reindex(columns=cols)

    try:
        if gsc_data:
            cols = "Cluster Name", "Keyword", "Cluster Size", "Cluster Volume", "Cluster Traffic", "Volume", "Traffic"
            df_matched = df_matched.reindex(columns=cols)
    except NameError:
        pass

In [506]:
# ------------ get the keyword with the highest search volume to replace the auto generated tag name with --------------

if col_len > 1:
    if parent_by_vol:
        df_matched['vol_max'] = df_matched.groupby(['Cluster Name'])['Volume'].transform(max)
        # this sort is mandatory for the renaming to work properly by floating highest values to the top of the cluster
        df_matched.sort_values(["Cluster Name", "Cluster Volume", "Volume"], ascending=[False, True, False], inplace=True)
        df_matched['exact_vol_match'] = df_matched['vol_max'] == df_matched['Volume']
        df_matched.loc[df_matched['exact_vol_match'] == True, 'highest_ranked_keyword'] = df_matched['Keyword']
        df_matched['highest_ranked_keyword'] = df_matched['highest_ranked_keyword'].fillna(method='ffill')
        df_matched['Cluster Name'] = df_matched['highest_ranked_keyword']
        del df_matched['vol_max']
        del df_matched['exact_vol_match']
        del df_matched['highest_ranked_keyword']
if adwords_check:
    df_matched = df_matched.rename(columns={"Volume": "Impressions", "Traffic": "Clicks", "Cluster Traffic": "Cluster Clicks (Sum)"})


In [507]:
# -------------------------------------- final output ------------------------------------------------------------------
# sort on cluster size
df_matched.sort_values(["Cluster Size", "Cluster Name", "Cluster Volume"], ascending=[False, True, False], inplace=True)

try:
    if gsc_data:
        df_matched.rename(
            columns={"Cluster Volume": "Cluster Impressions", "Cluster Traffic": "Cluster Clicks", "Traffic": "Clicks",
                     "Volume": "Impressions"}, inplace=True)
except NameError:
    pass

if col_len == 1:
    cols = "Cluster Name", "Keyword", "Cluster Size"
    df_matched = df_matched.reindex(columns=cols)

In [508]:
print(df_matched)

     Cluster Name                                            Keyword  \
527     ca domain                                          ca domain   
924     ca domain                                          ca domain   
2527    ca domain                                          at domain   
2798    ca domain                                             donain   
2825    ca domain                              domain purchase sites   
...           ...                                                ...   
6053   your hobby  how to pay yourself dividends from your corpor...   
6321   your hobby          how to pay yourself from your corporation   
7739   your hobby                        how to incorporate yourself   
8785   your hobby                 paying yourself from your business   
8799   your hobby                beyond yourself peanut butter dream   

      Cluster Size  Cluster Volume  Cluster KD (Median)  Cluster CPC (Median)  \
527            NaN          1620.0                 38.

In [509]:
# - add in intent markers
colname = df_matched.columns[1]
df_matched.loc[df_matched[colname].str.contains(info_filter), "Informational"] = "Informational"
df_matched.loc[df_matched[colname].str.contains(comm_invest_filter), "Commercial Investigation"] = "Commercial Investigation"
df_matched.loc[df_matched[colname].str.contains(trans_filter), "Transactional"] = "Transactional"
df_matched.loc[df_matched[colname].str.contains(alberta_filter), "Alberta"] = "Alberta"
#df_matched.loc[df_matched[colname].str.contains(cities_filter), "Cities"] = "Cities"
#df_matched.loc[df_matched[colname].str.contains(nearme_filter), "Near Me"] = "Near Me"
#df_matched.loc[df_matched[colname].str.contains(neighbor_filter), "Neighboring City"] = "Neighboring City"
df_matched.loc[df_matched[colname].str.contains(branded_filter), "Branded"] = "Branded"
#df_matched.loc[df_matched[colname].str.contains(insurance_filter), "Insurance"] = "Insurance"


In [510]:
# - add in cluster markers
#colname = df_matched.columns[1]
#df_matched.loc[df_matched[colname].str.contains(vehicle_filter), "Vehicle"] = "Vehicle"
#df_matched.loc[df_matched[colname].str.contains(property_filter), "Property"] = "Property"
#df_matched.loc[df_matched[colname].str.contains(pet_filter), "Pet"] = "Pet"
#df_matched.loc[df_matched[colname].str.contains(health_filter), "Life & Health"] = "Life & Health"
#df_matched.loc[df_matched[colname].str.contains(commercial_filter), "Business"] = "Business"

In [511]:
df_matched['detected_language'] = df_matched['Keyword'].apply(detect_language)

In [512]:
# Initiate the spell checker
spell = SpellChecker()

# Add custom words that are known to be correct
spell.word_frequency.load_words(['alberta', 'nuans', 'cra', 'nebs', 'provincially', 'vs', 'inc', 'llc', 'npo', 'tm', 'proprietorship', 'calgary', 'kpi', 'gst', 'edmonton', 'proprietorships', 'wcb', 'cpa'])

# Define a function to find misspellings, ignoring case
def find_misspellings(text):
    # Convert text to lower case
    words = text.lower().split()
    misspelled = spell.unknown(words)
    return ", ".join(misspelled)

# Apply the function to the 'Keyword' column (ensure this column name matches your DataFrame)
df_matched['Review'] = df_matched['Keyword'].apply(find_misspellings)


# Now, df_matched will have an additional column 'Review'


In [513]:
# find keywords from one column in another in any order and count the frequency
df_matched['Cluster Name'] = df_matched['Cluster Name'].str.strip()
df_matched['Keyword'] = df_matched['Keyword'].str.strip()

df_matched['First Word'] = df_matched['Cluster Name'].str.split(" ").str[0]
df_matched['Second Word'] = df_matched['Cluster Name'].str.split(" ").str[1]
df_matched['Total Keywords'] = df_matched['First Word'].str.count(' ') + 1

def ismatch(s):
    A = set(s["First Word"].split())
    B = set(s['Keyword'].split())
    return A.intersection(B) == A

df_matched['Found'] = df_matched.apply(ismatch, axis=1)

df_matched = df_matched. fillna('')

def ismatch(s):
    A = set(s["Second Word"].split())
    B = set(s['Keyword'].split())
    return A.intersection(B) == A
df_matched['Found 2'] = df_matched.apply(ismatch, axis=1)

# todo - document this algo. Essentially if it matches on the second word only, it renames the cluster to the second word
# clean up code nd variable names

df_matched.loc[(df_matched["Found"] == False) & (df_matched["Found 2"] == True), "Cluster Name"] = df_matched["Second Word"]
df_matched.loc[(df_matched["Found"] == False) & (df_matched["Found 2"] == False), "Cluster Name"] = "zzz_no_cluster_available"

# count cluster_size
df_matched['Cluster Size'] = df_matched['Cluster Name'].map(df_matched.groupby('Cluster Name')['Cluster Name'].count())
df_matched.loc[df_matched["Cluster Size"] == 1, "Cluster Name"] = "zzz_no_cluster_available"


df_matched = df_matched.sort_values(by="Cluster Name", ascending=True)

#delete the helper cols
del df_matched['First Word']
del df_matched['Second Word']
del df_matched['Total Keywords']
del df_matched['Found']
del df_matched['Found 2']

In [514]:
df_matched.to_csv('cleaned_clustered_ceywords.csv', index=False)
files.download("cleaned_clustered_ceywords.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>